# 1. Decorators 101

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
@deco
def target():
    print('running target()')

In [3]:
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner>

decorator는 다른 함수를 인수로 전달해서 호출하는 일반적인 callable과 동일

# 2. When Python Executes Decorators

function decorator는 module이 import되자마자 실행되지만, decorate된 function은 명시적으로 호출될 때만 실행된다

# 3. Decorator-Enhanced Strategy Pattern

In [5]:
promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

In [7]:
@promotion
def fidelity(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order(order):
    distinct_item = {item.product for item in order.cart}
    if len(distinct_item) >= 10:
        return order.total() * 0.7
    return 0

# 4. Variable Scope Rules

In [9]:
def f1(a):
    print(a)
    print(b)

In [10]:
b = 6
f1(3)

3
6


In [11]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [12]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


# 5. Closures

In [13]:
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [14]:
avg = Averager()
avg(10)

10.0

In [15]:
avg(11)

10.5

In [16]:
avg(12)

11.0

In [17]:
def make_average():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [18]:
avg = make_average()

In [19]:
avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg(12)

11.0

클로져는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수

# 6. The nonlocal Declaration

In [28]:
def make_average():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [30]:
avg = make_average()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [31]:
def make_average():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [32]:
avg = make_average()
avg(10)

10.0

# 7. Implementing a Simple Decorator

In [33]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return clocked

In [3]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [6]:
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [7]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)

**************************************** Calling snooze(.123)
[0.12817383s] snooze(0.123) -> None


In [8]:
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00007796s] factorial(2) -> 2
[0.00013208s] factorial(3) -> 6
[0.00018883s] factorial(4) -> 24
[0.00024295s] factorial(5) -> 120
[0.00029588s] factorial(6) -> 720
6! =  720


In [9]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [10]:
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling factorial(6)
[0.00000286s] factorial(1) -> 1
[0.00065804s] factorial(2) -> 2
[0.00111794s] factorial(3) -> 6
[0.00181699s] factorial(4) -> 24
[0.00233388s] factorial(5) -> 120
[0.00267386s] factorial(6) -> 720
6! =  720


# 8. Decorators in the Standard Library

In [14]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [15]:
print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00070190s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00018096s] fibonacci(2) -> 1
[0.00043201s] fibonacci(3) -> 2
[0.00141811s] fibonacci(4) -> 3
[0.00000095s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00014687s] fibonacci(2) -> 1
[0.00029612s] fibonacci(3) -> 2
[0.00000095s] fibonacci(0) -> 0
[0.00000191s] fibonacci(1) -> 1
[0.00047207s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000286s] fibonacci(1) -> 1
[0.00042796s] fibonacci(2) -> 1
[0.00078392s] fibonacci(3) -> 2
[0.00180888s] fibonacci(4) -> 3
[0.00225091s] fibonacci(5) -> 5
[0.00381613s] fibonacci(6) -> 8
8


In [12]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [13]:
print(fibonacci(6))

[0.00000119s] fibonacci(0) -> 0
[0.00000381s] fibonacci(1) -> 1
[0.00155187s] fibonacci(2) -> 1
[0.00000596s] fibonacci(3) -> 2
[0.00192690s] fibonacci(4) -> 3
[0.00000405s] fibonacci(5) -> 5
[0.00216508s] fibonacci(6) -> 8
8


# 9. Stacked Decorators

# 10. Parameterized Decorators